In [1]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

num_classes = 5
epochs = 100
train_list = list(range(5)) 
val_list = list(range(5)) 
test_list = list(range(5,10))
all_list = list(range(10))

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

#définition de la contrastive_loss
def contrastive_loss(y_true, y_pred):
    margin = 1
    return 0.5 * y_true * y_pred + 0.5 * (1 - y_true) * (K.maximum(margin - y_pred, 0))

#création des pairs positives et négatives 
def create_pairs(x, digit_indices, list_class):
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in list_class]) - 1
    for d in list_class:
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            # print(pairs)
            inc = random.randrange(1, len(list_class))
            dn = (d + inc) % len(list_class)
            z1, z2 = digit_indices[d][i], digit_indices[list_class[dn]][i]
            pairs += [[x[z1], x[z2]]]
            # print(pairs)
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


#création du modèle
def create_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation='relu')(x)
    return Model(input, x)


def compute_accuracy(y_true, y_pred):
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

#définition de la métrique accuracy pour le réseau siamois
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


# séparation du train et test 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]

# création des pairs sur le train et le test 
train_digit_indices = [np.where(y_train == i)[0] for i in all_list]
test_digit_indices = [np.where(y_test == i)[0] for i in all_list]

tr_pairs, tr_y = create_pairs(x_train, train_digit_indices, train_list)
te_pairs, te_y = create_pairs(x_test, test_digit_indices, val_list)

# network definition
network = create_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# les poids seront partagés sur les deux branches du siamese
output_a = network(input_a)
output_b = network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([output_a, output_b])
'''prediction = Dense(1,activation='sigmoid')(distance)'''
model = Model([input_a, input_b], distance)

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 58410 samples, validate on 9790 samples
Epoch 1/20





58410/58410 [==============================] - 6s 99us/step - loss: 0.0603 - accuracy: 0.9454 - val_loss: 0.0170 - val_accuracy: 0.9839
Epoch 2/20
58410/58410 [==============================] - 4s 60us/step - loss: 0.0132 - accuracy: 0.9870 - val_loss: 0.0085 - val_accuracy: 0.9872
Epoch 3/20
58410/58410 [==============================] - 3s 60us/step - loss: 0.0066 - accuracy: 0.9915 - val_loss: 0.0069 - val_accuracy: 0.9901
Epoch 4/20
58410/58410 [==============================] - 3s 60us/step - loss: 0.0050 - accuracy: 0.9940 - val_loss: 0.0056 - val_accuracy: 0.9916
Epoch 5/20
58410/58410 [==============================] - 3s 60us/step - loss: 0.0042 - accuracy: 0.9952 - val_loss: 0.0065 - val_accuracy: 0

In [0]:


# def create_nonseen_pairs(x, digit_indices):
#     pairs = []
#     labels = []
#     n = min([len(digit_indices[d]) for d in test_list]) - 1
#     for d in test_list:
#         for i in range(n):
#             z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
#             pairs += [[x[z1], x[z2]]]
#             inc = random.randrange(1, len(test_list))
#             dn = (d + inc) % len(test_list)
#             z1, z2 = digit_indices[d][i], digit_indices[dn][i]
#             pairs += [[x[z1], x[z2]]]
#             labels += [1, 0]
#     return np.array(pairs), np.array(labels)
test_list = list(range(5,7))
tt_pairs, tt_y = create_pairs(x_train, train_digit_indices, test_list)

'''digit_indices = [np.where(y_test == i)[0] for i in test_list]
tt_pairs, tt_y.append(create_nonseen_pairs(x_test, digit_indices))'''

from sklearn.metrics import accuracy_score

def get_one_shot_performance():
  batch, target = create_pairs(x_train, train_digit_indices, test_list)
  y_pred = model.predict([batch[:, 0], batch[:, 1]])
  acc = compute_accuracy(target, y_pred)
  # acc = np.mean(((y_pred.ravel() < 0.5) == target))
  return acc
  # batch, target = create_nonseen_pairs(tt_pairs, test_list)
  # y_pre = model.predict(batch)
  # pre = np.argmin(y_pre.reshape(-1,len(all_num_list)),axis=1)
  # return accuracy_score(target, pre)



In [3]:
get_one_shot_performance()

0.2169280442804428